# Function for cleaning the text

In [32]:
import pandas as pd
import nltk
import os
import csv
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import treebank,stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
import openpyxl
from openpyxl.utils import get_column_letter
0
def preprocess_tweets(text):

    #CONVERT TO LOWER CASE
    text=text.lower()

    #Remving hyperlinks
    text= re.sub(r"http\S+", "", text)

    #TOKENIZE THE WORDS
    words=word_tokenize(text,"english")

    #Removing all the stop words
    words=[w for w in words if w not in stopwords.words('english')]

    stemmer=PorterStemmer()
    words=[stemmer.stem(w) for w in words]
    #Lemmatizing the words- Extracting the root word
    #Creating instance for WordNetLemmatizer in nltk
    lemmatizer=WordNetLemmatizer()
    words =[lemmatizer.lemmatize(w) for w in words]

    #REMOVE ALL THE PUNCTUATIONS AND NUMERIC VALUES
    cleaned_words=[w for w in words if w.isalpha()]
    # print(cleaned_words)
    # print(len(cleaned_words))

    #Assiging tags
    pos_tags=nltk.pos_tag(cleaned_words)
    # print(pos_tags)

    cleaned_text=" ".join(cleaned_words)
    return cleaned_text





# Function to create the excel file containing the cleaned tweets of 2 timeframes

In [55]:
def cleaning_tweets(filename):
    # open the input Excel file 
    workbook = openpyxl.load_workbook(filename)

    # create a new workbook to write the output
    output_workbook = openpyxl.Workbook()
    output_workbook.remove(output_workbook.active)

    # process each sheet in the input file and write to a new sheet in the output file
    for sheet_name in workbook.sheetnames:
        worksheet = workbook[sheet_name]
        output_worksheet = output_workbook.create_sheet(title=f'cleaned_{sheet_name}')

        # write the header row
        header = ['screen_name', 'cleaned_text']
        for col_num, column_title in enumerate(header, 1):
            column_letter = get_column_letter(col_num)
            output_worksheet[f"{column_letter}1"] = column_title

        # iterate over the rows in the original sheet
        for row_num in range(2, worksheet.max_row + 1):
            screen_name = worksheet.cell(row=row_num, column=4).value
            text = worksheet.cell(row=row_num, column=2).value
            cleaned_text = preprocess_tweets(text)

            # write the data to the output sheet
            output_worksheet.cell(row=row_num, column=1, value=screen_name)
            output_worksheet.cell(row=row_num, column=2, value=cleaned_text)
    
    filename = filename.split('/')[-1] 
    # save the output Excel file
    output_workbook.save(f'C:/Users/ektag/Minor2/sentiment_analysis/cleaned_filtered_and_sorted_tweets/cleaned_{filename}')

In [42]:
import os
directory="C:/Users/ektag/Minor2/sentiment_analysis/filtered_and_sorted_tweets/"
for filename in os.listdir(directory):
    f=os.path.join(directory,filename)
    if os.path.isfile(f) and filename.endswith('.xlsx'):
        cleaning_tweets(directory+filename)